# Sagemaker

In [ ]:
#wandb.finish()
!pip install --upgrade pip
!pip install "transformers==4.30.2" 
!pip install "datasets[s3]==2.13.0" 
!pip install sagemaker --upgrade 
!pip install wandb
#!pip install "transformers==4.30.2" --upgrade
#!pip3 install git+https://github.com/huggingface/transformers
!pip install torch 

In [ ]:
from getpass import getpass
import os

import sagemaker
import boto3
import torch
##import wandb
from huggingface_hub import login
from transformers import LlamaTokenizer
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
import time
from datetime import datetime
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder
import wandb

hf_token=None
wandb_token=None

hf_token = getpass('input hf token')
wandb_token = getpass('input wandb token')

login(token=hf_token)
wandb.login(key=wandb_token)

In [ ]:
#sess = sagemaker.Session()
#sagemaker_session_bucket='llms-hf'
#
#iam = boto3.client('iam')
#role = iam.get_role(RoleName='Sagemaker-DataScientist')['Role']['Arn']
##role = iam.get_role(RoleName='sagemaker')['Role']['Arn']
#
##sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
#sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
#sagemaker_session_bucket = sess.default_bucket()
#artifact_dir='artifacts/combined_dataset:v1'

In [ ]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
#sagemaker_session_bucket='llms-hf'
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

    

iam = boto3.client('iam')
role = iam.get_role(RoleName='Sagemaker-DataScientist')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [39]:
sagemaker_session_bucket

'sagemaker-us-east-1-338563806291'

In [ ]:
now = datetime.now()
time_stamp = now.strftime("%m.%d.%y-%H.%M.%S")
with wandb.init(project='SFT_training_DM',
                entity='ft-llmmm',
                job_type='download_data',
                name=f'download_combined_data_{time_stamp}') as run:
    
    artifact = run.use_artifact(f'ft-llmmm/ELI5_analysis/llama_QA_tokenized:latest', type='dataset')
    artifact_full_dir = artifact.download()

In [ ]:
now = datetime.now()
time_stamp = now.strftime("%m.%d.%y-%H.%M.%S")
with wandb.init(project='SFT_training_DM',
                entity='ft-llmmm',
                job_type='download_data',
                name=f'download_combined_data_{time_stamp}') as run:
    artifact_dir = {}
    
    
    for key in [1024,2048]:
    
        artifact = run.use_artifact(f'ft-llmmm/ELI5_analysis/llama_QA_tokenized_{key}:latest', type='dataset')
        artifact_dir[key] = artifact.download()

In [ ]:
run_name

In [42]:
model_id = "meta-llama/Llama-2-7b-hf" # sharded weights
model_name = model_id.split('/')[-1]
ds_name = 'combined_1024'
training_input_path = f's3://{sagemaker_session_bucket}/{ds_name}'

# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

run_name = f'{model_name}_{ds_name}_qlora'
run_name += f'_{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# enables spot training
use_spot_instances = True
# max time including spot start + training time
max_wait = 72*60*60
# expected training time
max_run = 48*60*60

run_name = f'{model_name}_{ds_name}_qlora'
job_name = f'{model_name}_{ds_name}-qlora'.replace('_','-')

if use_spot_instances:
    job_name += '-spot'
    run_name += '-spot'

run_name += f'__{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

checkpoint_s3_uri = f's3://{sess.default_bucket()}/{job_name}/checkpoints'
    
max_seq_length = 4096

if int(ds_name[-4:]) in [1024,2048]:
    max_seq_length = int(ds_name[-4:])
    
hyperparameters = {
    'model_id': model_id,
    'repo_id': f'{model_name}-{ds_name}',
    'dataset_path': '/opt/ml/input/data/training',
    'hf_token': hf_token,
    'wandb_token': wandb_token,
    'report_to_wandb': 1,
    'epochs': 3,
    'max_steps': -1,
    'per_device_train_batch_size': 16,
    'per_device_eval_batch_size': 16,
    'gradient_accumulation_steps': 8,
    'lr': 2e-5,
    'merge_weights': 0,
    'entity': 'ft-llmmm',
    'project_name': 'SFT_training_dm',
    'hub_strategy': 'every_save',
    'run_name': run_name,
    'torch_compile': 0,
    'gradient_checkpointing': 1,
    'optim': 'paged_adamw_8bit',
    'group_by_length': 1,
    #'fsdp': '"full_shard auto_wrap"',
    #'fsdp_transformer_layer_cls_to_wrap': "LlamaDecoderLayer",
    #'lora_r'
}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = './',         # directory which includes all the files needed for training
    instance_type        = 'ml.p4d.24xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.28',            # the transformers version used in the training job
    #transformers_version = '4.30',
    pytorch_version      = '2.0',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
    use_spot_instances = use_spot_instances,
    max_wait = max_wait,
    max_run=max_run,
    checkpoint_s3_uri = checkpoint_s3_uri
)

In [44]:
max_seq_length

1024

In [43]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: Llama-2-7b-hf-combined-1024-qlora-spot-2023-08-19-20-46-55-882


2023-08-19 20:50:00 Starting - Starting the training job
2023-08-19 20:50:00 Pending - Training job waiting for capacity.............................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [ ]:
sagemaker-us-east-1-338563806291


# EC2 + FlashAttention

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

In [ ]:
!pip install wandb
import wandb

In [ ]:
with wandb.init(project='SFT_Training_dm',
                 entity='ft-llmmm',
                 job_type='download_data',
                 name='download_SFT_EC2') as run:
    artifact = run.use_artifact('ft-llmmm/ELI5_analysis/llama_QA_tokenized:v2', type='dataset')
    artifact_dir = artifact.download()
    
    print(f'artifact saved to {artifact_dir}')

In [ ]:
!pip install datasets
import datasets

In [ ]:
ds_QA_SFT = datasets.load_from_disk(f'./{artifact_dir}')
ds_wiki = ds_QA_SFT.filter(lambda x:x['source']=='simple_wiki')
ds_ELI5 = ds_QA_SFT.filter(lambda x:x['source']=='ELI5')

ds_wiki.save_to_disk('./data/ds_wiki')
ds_ELI5.save_to_disk('./data/ds_ELI5')

In [ ]:
import time
from datetime import datetime

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf" # sharded weights
model_name = model_id.split('/')[-1]
ds_name = 'wiki'

if ds_name == 'combined':
    dataset_path = artifact_dir
else:
    dataset_path = f'./data/ds_{ds_name}'

run_name = f'{model_name}_{ds_name}_qlora_flash'
run_name += f'__{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

output_dir = f'./{model_name}_{ds_name}/models'
logging_dir = f'./{model_name}_{ds_name}/logs'
repo_id = f'{model_name}-{ds_name}-flash'
report_to_wandb = 1
epochs = 3
max_steps = -1
per_device_train_batch_size = 8
per_device_eval_batch_size = 8
gradient_accumulation_steps = 16
lr = 2e-4
merge_weights = 0
entity = 'ft-llmmm'
project_name = 'SFT_training_dm'
hub_strategy = 'every_save'
torch_compile = 0
gradient_checkpointing = 1
optim = 'paged_adamw_8bit'
group_by_length = 0

In [ ]:
logging_dir

In [ ]:
wandb_token

In [ ]:
import os

In [ ]:
hf_token = os.environ['HUGGINGFACE_TOKEN']
wandb_token = os.environ['WANDB_API_KEY']

In [ ]:
optim

In [ ]:
%python ./run_clm.py \
--output_dir './Llama-2-7b-hf_wiki/models' \
--logging_dir './Llama-2-7b-hf_wiki/logs' \
--model_id 'meta-llama/Llama-2-7b-hf' \
--dataset_path './data/ds_wiki' \
--run_name 'Llama-2-7b-hf_wiki_qlora_no_flash_test' \
--repo_id 'Llama-2-7b-hf-wiki-flash' \
--report_to_wandb 1 \
--epochs 3 \
--max_steps -1 \
--per_device_train_batch_size 8 \
--per_device_eval_batch_size 8 \
--gradient_accumulation_steps 16 \
--lr 2e-4 \
--merge_weights 0 \
--entity 'ft-llmmm' \
--project_name 'SFT_training_dm' \
--hub_strategy 'every_save' \
--torch_compile 0 \
--gradient_checkpointing 1 \
--optim 'paged_adamw_8bit' \
--group_by_length 1 \
--hf_token hf_token \
--wandb_token wandb_token \
--use_flash_attention 1